In [132]:
import time
import requests
from time import sleep
import re, requests, csv
from bs4 import BeautifulSoup as bs
from selenium.webdriver.common.keys import Keys
import pandas as pd
from selenium import webdriver
keys = Keys()

In [133]:
from selenium.webdriver.common.by import By

In [134]:
next_btn = []
for i in range(2, 12):
    next_btn.append(f'a:nth-child({i})')
next_btn.append("a.fAUKm1ewwo._2Ar8-aEUTq")

In [213]:
def get_reviews(address, stop):
    driver = webdriver.Chrome()
    chromeOptions = webdriver.ChromeOptions()
    chromeOptions.add_argument("--hide-scrollbars")
    driver.get(address)
    page = requests.get(address)

    # stop이 총 리뷰 수보다 많다면, 총 리뷰 수로 계산하기

    print(f"{address} 페이지 리뷰 크롤링을 시작합니다.")

    count = 0

    review_list = []

    driver.implicitly_wait(3)
    
    # 리뷰 보기 버튼 클릭
    el = driver.find_element(By.CSS_SELECTOR, '#content > div > div.z7cS6-TO7X > div._27jmWaPaKy > ul > li:nth-child(2) > a')
    actions = webdriver.ActionChains(driver).move_to_element(el).click().perform()
    print("리뷰 창으로 넘어감")
    time.sleep(10)

    # 리뷰 낮은 순 보기 버튼 클릭
    el = driver.find_element(By.XPATH, '//*[@id="REVIEW"]/div/div[3]/div[1]/div[1]/ul/li[4]/a')
    actions = webdriver.ActionChains(driver).move_to_element(el).perform()
    actions2 = webdriver.ActionChains(driver).send_keys(Keys.ARROW_DOWN).perform()
    el.send_keys(Keys.ENTER)
    print("리뷰 낮은 순으로 정렬")
    time.sleep(3)

    while (len(review_list) < stop):
        if (count % 20 == 0):
            el = driver.find_element(By.CSS_SELECTOR, "#REVIEW > div > div._180GG7_7yx > div.cv6id6JEkg > div > div > " + str(next_btn[(count // 20) % 11]) + '') # 리뷰 인덱스 페이지 이동
            actions = webdriver.ActionChains(driver).move_to_element(el)
            driver.implicitly_wait(3)
            actions.perform()
            el.click()
            time.sleep(2)
            html = driver.page_source
            soup = bs(html, "html.parser")
            reviews = soup.find_all('div', class_='YEtwtZFLDz')
            print(f"불러온 리뷰 : {len(reviews)}개")
        review = reviews[count % 20].text
        review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]', "", review)
        if ("더보기" in review):
            review = soup.find_all('div', class_="YEtwtZFLDz.vlXMQEAtPR")
            review = review[count % 20].text
            review = re.sub('[^#0-9a-zA-Zㄱ-ㅣ가-힣 ]', "", review)
        review_list.append(review)
        count = count + 1
        print(f'{count}번째 리뷰 가져옴')
    driver.quit()
    return review_list

In [197]:
PAGES = ['https://brand.naver.com/skinfood/products/7926935977',
        'https://smartstore.naver.com/kikoluggage/products/6399548845?n_media=11068&n_query=%EA%B8%B0%EB%82%B4%EC%9A%A9%EC%BA%90%EB%A6%AC%EC%96%B4&n_rank=1&n_ad_group=grp-a001-02-000000033272787&n_ad=nad-a001-02-000000235065323&n_campaign_type=2&n_mall_id=shc2016&n_mall_pid=6399548845&n_ad_group_type=2&n_match=3'
         ]

In [214]:
review_pages = {}
for i in PAGES:
    review_pages[i] = get_reviews(i, 30)

https://brand.naver.com/skinfood/products/7926935977 페이지 리뷰 크롤링을 시작합니다.
리뷰 창으로 넘어감
리뷰 낮은 순으로 정렬
불러온 리뷰 : 20개
1번째 리뷰 가져옴
2번째 리뷰 가져옴
3번째 리뷰 가져옴
4번째 리뷰 가져옴
5번째 리뷰 가져옴
6번째 리뷰 가져옴
7번째 리뷰 가져옴
8번째 리뷰 가져옴
9번째 리뷰 가져옴
10번째 리뷰 가져옴
11번째 리뷰 가져옴
12번째 리뷰 가져옴
13번째 리뷰 가져옴
14번째 리뷰 가져옴
15번째 리뷰 가져옴
16번째 리뷰 가져옴
17번째 리뷰 가져옴
18번째 리뷰 가져옴
19번째 리뷰 가져옴
20번째 리뷰 가져옴
불러온 리뷰 : 20개
21번째 리뷰 가져옴
22번째 리뷰 가져옴
23번째 리뷰 가져옴
24번째 리뷰 가져옴
25번째 리뷰 가져옴
26번째 리뷰 가져옴
27번째 리뷰 가져옴
28번째 리뷰 가져옴
29번째 리뷰 가져옴
30번째 리뷰 가져옴
https://smartstore.naver.com/kikoluggage/products/6399548845?n_media=11068&n_query=%EA%B8%B0%EB%82%B4%EC%9A%A9%EC%BA%90%EB%A6%AC%EC%96%B4&n_rank=1&n_ad_group=grp-a001-02-000000033272787&n_ad=nad-a001-02-000000235065323&n_campaign_type=2&n_mall_id=shc2016&n_mall_pid=6399548845&n_ad_group_type=2&n_match=3 페이지 리뷰 크롤링을 시작합니다.
리뷰 창으로 넘어감
리뷰 낮은 순으로 정렬
불러온 리뷰 : 20개
1번째 리뷰 가져옴
2번째 리뷰 가져옴
3번째 리뷰 가져옴
4번째 리뷰 가져옴
5번째 리뷰 가져옴
6번째 리뷰 가져옴
7번째 리뷰 가져옴
8번째 리뷰 가져옴
9번째 리뷰 가져옴
10번째 리뷰 가져옴
11번째 리뷰 가져옴
12번째 리뷰 가져옴
13번째 리뷰 가져옴
14번째 리뷰 가져옴
15번째 리뷰

In [216]:
print(review_pages[PAGES[0]])

['한달사용기도토리 패드는 여드름 피부 적합하지 않습니다 끈적이는 느낌이 많아서 당근패드와 같은 산뜻함이 없어서 여드름에 악영향인 것 같아 사용을 중단했습니다', '한달사용기재주문은 안할 겁니다', '한달사용기정말 별로예요 당근패드가 나아요 솜털이 묻어나서 할때마다 피부 정돈을 또 해야해요', '한달사용기당근패드 사면서 도토리도 사 봤는데이거 별루예요보플떼기 심해요ㅠㅠ닦아내면 피부 쓸리고 아프고따갑고꾹꾹 눌러 흡수 시켜도 솜 보플 일어나 얼굴에 덕지덕ㅈ 묻어요', '저 뿐만이 아니라 다들 사은품 못 받으셨네요종종 할인 되서 올라오는 가격에 사은품 증정을 크게 내세워 6만원이상 구매 했는데 상술이였네요네 앞으로 여기거 안쓰면 됩니다비슷한 핫딜가로 종종 구매 했을 때는 하다못해 샘플증정이나 케이스라도 주었는데이번에는 배송지연도 엄청 기다리게 하고 구매한 상품 6통 택배 상자에 제각각 굴러다녀 상자조차 찌그러졌네요하다못해 백화점에서 썬크림하나 사도 주는 샘플 포장도 참 어이없고 브랜드 데이에 정 떨어지게 해주시니 안쓰고 안사면 됩니다', '배송엄청느리네요 15일주문 이제받았고 사은품 품절이라며 주지도않네요', '한달사용기와 이거 쓰고 저 좁쌀여드름 진짜 많이나고 피부 완전 뒤집어졌오요 믿고 구매했는데 대대실망', '다음날 직진배송이라 구매했는데 한참을 안와서 문의하니 배송사고 ㅜ ㅜ연락도안주시고 2주만에 받았는데 그저그래요', '한달사용기얼굴에 자극적이라 바디에 닦아봤는데패드가 튼튼하지않아요', '도토리패드는 당근패드와 다르게 두툼해요 반 잘라서 써야 하지 뜯을려고 하니까 안뜯어져서 한장 올려서 써요 두툼하고 무슨 옥수수재질로 만든거같아요 그리고 부드럽지 않아요 역시 당근패드가 훨 나아요 저렴하게 샀지만 도토리패드 궁금했는데 해결됐네요', '한달사용기어우 이거 제가 받은것만 그랬는지 몰라도 액체도 안에 없고 솜도 다 일어나는 재질이라 쓸고나면 얼굴에 솜이 조금씩 묻어서 너무너무 화났어요 촉촉하지도 않아서 다시는 재구매안하려고요ㅠ', '한달사용기당근패드가 더 촉촉하고